In [ ]:
import os
import json
import pandas as pd
from glob import glob
import numpy

# Define the base directories
base_dirs = ['train', 'eval', 'test']

for i in range(len(base_dirs)):
    base_dirs[i] = 'data/methods2test_data/' + base_dirs[i]
# List to collect all data (rows)
data = []

total_exmaples = 0

is_broken = False
base_dir = 'data/methods2test_data/train'
# Iterate over all subdirectories (examples) in the base directory
for example_dir in os.listdir(base_dir):
    example_path = os.path.join(base_dir, example_dir)
    
    # Ensure we're dealing with directories (i.e., each example is its own directory)
    if os.path.isdir(example_path):
        # Use glob to get all JSON files in the example directory
        json_files = glob(os.path.join(example_path, '*.json'))
        
        # Iterate over the JSON files
        for json_file in json_files:
            # Read the JSON data
            with open(json_file, 'r') as f:
                json_data = json.load(f)
            
            # Add the JSON data to the data list (can modify if additional info is needed)
            data.append(json_data)
            total_exmaples+=1
            if(total_exmaples>=100):
                print("breaking")
                is_broken = True
                break
    if(is_broken):
        break
# Convert the collected data into a pandas DataFrame
df = pd.DataFrame(data)

# Optional: Inspect the first few rows
#print(df.head())